In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

In [2]:
data_prediction = pd.read_parquet('prediction_index.parquet')
data_prediction

,week,customer,product
0,90,0,0
1,90,0,1
2,90,0,2
3,90,0,3
4,90,0,4
...,...,...,...
499995,90,1999,245
499996,90,1999,246
499997,90,1999,247
499998,90,1999,248


In [3]:
# samll dataset (somthing going wrong on mismatching the index and the rows)

# data_basket_s = pd.read_parquet('baskets-s.parquet')
# print (data_basket_s)

# data_coupons_s = pd.read_parquet('coupons-s.parquet')
# print (data_coupons_s)

In [4]:
# middle dataset

# data_basket_m = pd.read_parquet('baskets-m.parquet')
# print (data_basket_m)

# data_coupons_m = pd.read_parquet('coupons-m.parquet')
# print (data_coupons_m)

In [5]:
# large dataset (this is the dataset we are currently browsing. all the three datasets should be the same, but different sizes between them)

data_basket_l = pd.read_parquet('baskets-l.parquet')
print (data_basket_l)

data_coupons_l = pd.read_parquet('coupons-l.parquet')
print (data_coupons_l)

          week  customer  product  price
0            0         0       71    629
1            0         0       91    605
2            0         0      116    715
3            0         0      123    483
4            0         0      157    592
...        ...       ...      ...    ...
68841593    89     99999      143    470
68841594    89     99999      158    566
68841595    89     99999      186    499
68841596    89     99999      204    496
68841597    89     99999      225    602

[68841598 rows x 4 columns]
          week  customer  product  discount
0            0         0       35        35
1            0         0      193        40
2            0         0       27        30
3            0         0      177        35
4            0         0        5        30
...        ...       ...      ...       ...
44999995    89     99999       62        40
44999996    89     99999      110        10
44999997    89     99999       37        40
44999998    89     99999      155      

In [6]:
# merging the basket and coupons dataframe together
merged_data_outer = pd.merge(data_basket_l, data_coupons_l, on=['customer', 'week', 'product'], how='outer')

merged_data_outer

,week,customer,product,price,discount
0,0,0,71,629.0,NaN
1,0,0,91,605.0,NaN
2,0,0,116,715.0,NaN
3,0,0,123,483.0,NaN
4,0,0,157,592.0,NaN
...,...,...,...,...,...
111133550,89,99998,167,NaN,20.0
111133551,89,99999,62,NaN,40.0
111133552,89,99999,37,NaN,40.0
111133553,89,99999,155,NaN,25.0


In [7]:
merged_data_outer_sort = merged_data_outer.sort_values(by = ['week', 'product', 'customer'])
merged_data_outer_sort.head ## checking the data by yourself

<bound method NDFrame.head of            week  customer  product  price  discount
68841659      0        13        0    NaN      20.0
269           0        35        0  688.0       NaN
68841965      0        78        0    NaN      10.0
68842259      0       138        0    NaN      15.0
68842429      0       174        0    NaN      20.0
...         ...       ...      ...    ...       ...
68841063     89     99925      249  499.0       NaN
111133241    89     99932      249    NaN      20.0
68841344     89     99967      249  499.0       NaN
68841476     89     99984      249  499.0       NaN
111133523    89     99993      249    NaN      20.0

[111133555 rows x 5 columns]>

In [8]:
# this is the dataframe excluding from the previous merging
merged_data_inner = pd.merge(data_basket_l, data_coupons_l, on=['customer', 'week', 'product'], how='inner')

merged_data_inner

,week,customer,product,price,discount
0,0,5,221,442,35
1,0,7,61,739,10
2,0,8,74,584,25
3,0,9,194,382,30
4,0,11,245,439,20
...,...,...,...,...,...
2708038,89,99987,205,251,40
2708039,89,99988,192,384,30
2708040,89,99992,72,512,35
2708041,89,99995,179,474,20


In [9]:
# calculating and checking the orginal price that is price before the discount
new_data = []
for index, row in merged_data_inner.iterrows():
    price = row['price']
    discount = row['discount']
    original_price = price / ((100 - discount) / 100)
    new_data.append(original_price)

merged_data_inner['original_price'] = new_data

merged_data_inner.sort_values(by = ['week', 'product'])

merged_data_inner.head

<bound method NDFrame.head of          week  customer  product  price  discount  original_price
0           0         5      221    442        35      680.000000
1           0         7       61    739        10      821.111111
2           0         8       74    584        25      778.666667
3           0         9      194    382        30      545.714286
4           0        11      245    439        20      548.750000
...       ...       ...      ...    ...       ...             ...
2708038    89     99987      205    251        40      418.333333
2708039    89     99988      192    384        30      548.571429
2708040    89     99992       72    512        35      787.692308
2708041    89     99995      179    474        20      592.500000
2708042    89     99999      110    581        10      645.555556

[2708043 rows x 6 columns]>